In [ ]:
# import simulation stuff
import conf_solo12_trot_step_adjustment_full_kinematics as conf
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)
# rmodel = simulator.robot.pin_robot.model
# rdata = simulator.robot.pin_robot.data
# print(rdata.oMf[rmodel.getFrameId(conf.ee_frame_names[0])])
# print(rdata.oMf[rmodel.getFrameId(conf.ee_frame_names[1])])
# print(rdata.oMf[rmodel.getFrameId(conf.ee_frame_names[2])])
# print(rdata.oMf[rmodel.getFrameId(conf.ee_frame_names[3])])
# # initial feet positions
# flFootPos = [0.1946 , 0.14695, 0.]
# frFootPos = [ 0.1946, -0.14695, 0.]
# hlFootPos = [-0.1946, 0.14695,  0.]
# hrFootPos = [-0.1946, -0.14695, 0.]
# w = 0.5*conf.step_adjustment_bound
# step_length = conf.gait['stepLength']
# # 0
# simulator.build_one_stepstone(start_pos=(flFootPos[0]-w, flFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(frFootPos[0]-w, frFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hlFootPos[0]-w, hlFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hrFootPos[0]-w, hrFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# # 1
# simulator.build_one_stepstone(start_pos=(flFootPos[0]+step_length-w, flFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(frFootPos[0]+step_length-w, frFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hlFootPos[0]+step_length-w, hlFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hrFootPos[0]+step_length-w, hrFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# # 2
# simulator.build_one_stepstone(start_pos=(flFootPos[0]+2*step_length-w, flFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(frFootPos[0]+2*step_length-w, frFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hlFootPos[0]+2*step_length-w, hlFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hrFootPos[0]+2*step_length-w, hrFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# # 3
# simulator.build_one_stepstone(start_pos=(flFootPos[0]+3*step_length-w, flFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(frFootPos[0]+3*step_length-w, frFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hlFootPos[0]+3*step_length-w, hlFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hrFootPos[0]+3*step_length-w, hrFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# # 4
# simulator.build_one_stepstone(start_pos=(flFootPos[0]+4*step_length-w, flFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(frFootPos[0]+4*step_length-w, frFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hlFootPos[0]+4*step_length-w, hlFootPos[1], 0.0), orientation=(0., 0., 0, 1))
# simulator.build_one_stepstone(start_pos=(hrFootPos[0]+4*step_length-w, hrFootPos[1], 0.0), orientation=(0., 0., 0, 1))

In [ ]:
# import models and solvers
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model import CentroidalPlusLegKinematicsCasadiModel
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
import numpy as np

# DDP warm-start
wbd_model = WholeBodyModel(conf)
ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=False, WARM_START=False)
ddp_planner.solve()
ddp_sol = ddp_planner.get_solution_trajectories()
centroidal_warmstart = ddp_sol['centroidal']
q_warmstart = ddp_sol['jointPos']
qdot_warmstart = ddp_sol['jointVel']
x_warmstart = []
u_warmstart = []
rmodel, rdata = conf.rmodel, conf.rdata
for k in range(len(centroidal_warmstart)):
    x_warmstart.append(
        np.concatenate(
            [centroidal_warmstart[k],
             q_warmstart[k], 
             qdot_warmstart[k]]
            )
        )
    u_warmstart.append(np.concatenate([np.zeros(30)]))
# # nominal traj-opt
model_nom = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=False)
solver_nom = CentroidalPlusLegKinematicsAcadosSolver(
    model_nom, x_warmstart, u_warmstart, MPC=False
)

# import matplotlib.pylab as plt
# f_ref = np.array(solver_nom.swing_feet_tasks).reshape(len(solver_nom.swing_feet_tasks), 12) 
# time = np.arange(0, np.round(f_ref.shape[0]*conf.dt, 2), conf.dt)
# for contact in conf.ee_frame_names:
#     plt.rc('font', family ='serif')
#     contact_name = contact[0:2]
#     if contact_name == 'FL':
#       f_scp_ref = f_ref[:, 0:3]
#     elif contact_name == 'FR':
#       f_scp_ref = f_ref[:, 3:6] 
#     elif contact_name == 'HL':
#       f_scp_ref = f_ref[:, 6:9] 
#     elif contact_name == 'HR':
#       f_scp_ref = f_ref[:, 9:12]
#     fig, (fx, fy, fz) = plt.subplots(3, 1, sharex=True) 
#     fx.step(time, f_scp_ref[:, 0], label=' SCP forces nom (N)')
#     fx.legend()
#     fx.set_title('force x')
#     fy.step(time, f_scp_ref[:, 1])
#     fy.set_title('force y')
#     fz.step(time, f_scp_ref[:, 2])
#     fz.set_title('force z')
#     fz.set_xlabel('Time (s)', fontsize=14)
#     fig.suptitle('contact forces of '+ str(contact[0:2]))
x_nom, u_nom = solver_nom.solve()

In [ ]:
import matplotlib.pylab as plt
import pinocchio as pin
import numpy as np
import utils

# visualize motion in meshcat
dt = conf.dt
dt_ctrl = conf.dt_ctrl
N_ctrl =  int(dt/dt_ctrl)
# kinematic functions
ee_fk = model_nom.casadi_model.ee_fk
ee_jacobians = model_nom.casadi_model.ee_jacobians
# load robot in meshcat viewer
if conf.WITH_MESHCAT_DISPLAY:
    viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model)
    try:
        viz.initViewer(open=True)
    except ImportError as err:
        print(err)
        sys.exit(0)
    viz.loadViewerModel()
# add nominal contact surfaces
s = conf.step_adjustment_bound
for i, contacts in enumerate(conf.contact_sequence):
    for contact_idx, contact in enumerate(contacts):
        if contact.ACTIVE:
            t = contact.pose.translation
            if contact.CONTACT == 'FR' or contact.CONTACT == 'FL':
                utils.addViewerBox(
                    viz, 'world/contact'+str(i)+str(contact_idx), 
                    s, s, 0., [1., .2, .2, .5]
                    )
            if contact.CONTACT == 'HR' or contact.CONTACT == 'HL':
                utils.addViewerBox(
                    viz, 'world/contact'+str(i)+str(contact_idx),
                     s, s, 0., [.2, .2, 1., .5]
                    )       
            utils.applyViewerConfiguration(
                viz, 'world/contact'+str(i)+str(contact_idx), 
                [t[0], t[1], 0., 1, 0, 0, 0]
                )

f_total = []
q  = []
qdot = []
qddot = []
for traj_idx in range(conf.N-1):
    q_base = np.array(
            model_nom.casadi_model.q_plus(
                x_warmstart[traj_idx][3:7], x_nom[traj_idx, 12:15]*dt
                )
            ).squeeze()
    q_base_next = np.array(
            model_nom.casadi_model.q_plus(
                x_warmstart[traj_idx+1][3:7], x_nom[traj_idx+1, 12:15]*dt
                )
            ).squeeze() 
    # get generalized positions
    q_des = np.concatenate(
        [
            x_nom[traj_idx, 9:12], 
            q_base,
            x_nom[traj_idx, 15:27]
        ]
    )
    q_des_next = np.concatenate(
        [
            x_nom[traj_idx+1, 9:12], 
            q_base_next,
            x_nom[traj_idx+1, 15:27]]
        )
    # get generalized velocities
    v_des = x_nom[traj_idx, 27:]
    v_des_next = x_nom[traj_idx+1, 27:]
    
    # get generalized velocities
    a_des = u_nom[traj_idx, 12:]
    a_des_next = u_nom[traj_idx+1, 12:]

    # interpolate solution for low-level torque control
    q_interpol, qdot_interpol, qddot_interpol, f_interpol = utils.interpolate_one_step(
        rmodel, dt, dt_ctrl, 
        q_des, q_des_next, 
        v_des, v_des_next,
        a_des, a_des_next,
        u_nom[traj_idx, :12],  u_nom[traj_idx+1, :12]
    )
    for ctrl_idx in range(N_ctrl): 
        f_total += [f_interpol[ctrl_idx]]
        q += [q_interpol[ctrl_idx]]
        qdot += [qdot_interpol[ctrl_idx]]
        qddot += [qddot_interpol[ctrl_idx]]
        viz.display(q_interpol[ctrl_idx, :])
# plot optimized contact forces
f_total = np.array(f_total) 
q = np.array(q)
qdot = np.array(qdot)
qddot = np.array(qddot)
time = np.arange(0, np.round((f_total.shape[0])*dt_ctrl, 2), dt_ctrl)
for contact in conf.ee_frame_names:
    plt.rc('font', family ='serif')
    contact_name = contact[0:2]
    if contact_name == 'FL':
      f_scp_nom = f_total[:, 0:3]
    elif contact_name == 'FR':
      f_scp_nom = f_total[:, 3:6] 
    elif contact_name == 'HL':
      f_scp_nom = f_total[:, 6:9] 
    elif contact_name == 'HR':
      f_scp_nom = f_total[:, 9:12]
    fig, (fx, fy, fz) = plt.subplots(3, 1, sharex=True) 
    fx.step(time, f_scp_nom[:, 0], label=' contact forces nom (N)')
    fx.legend()
    fx.set_title('force x')
    fy.step(time, f_scp_nom[:, 1])
    fy.set_title('force y')
    fz.step(time, f_scp_nom[:, 2])
    fz.set_title('force z')
    fz.set_xlabel('Time (s)', fontsize=14)
    fig.suptitle('contact forces of '+ str(contact[0:2]))

# plot joint positions,velocities and accelerations
time = np.arange(0, np.round((q.shape[0])*dt_ctrl, 2), dt_ctrl)
for contact in conf.ee_frame_names:
    plt.rc('font', family ='serif')
    contact_name = contact[0:2]
    if contact_name == 'FL':
        q_plot = q[:, 7:10]
        qdot_plot = qdot[:, 6:9]
        qddot_plot = qddot[:, 6:9]
    elif contact_name == 'FR':
        q_plot = q[:, 10:13]
        qdot_plot = qdot[:, 9:12]
        qddot_plot = qddot[:, 9:12]
    elif contact_name == 'HL':
        q_plot = q[:, 13:16]
        qdot_plot = qdot[:, 12:15]
        qddot_plot = qddot[:, 12:15]
    elif contact_name == 'HR':
        q_plot = q[:, 16:19]
        qddot_plot = qdot[:, 15:18]
        qddot_plot = qddot[:, 15:18]
    # joint positions
    fig1, (HAA, HFE, KFE) = plt.subplots(3, 1, sharex=True)
    fig1.suptitle(str(contact[0:2])+ ' joint positions (rad)')
    HAA.set_title('HAA')
    HAA.step(time, q_plot[:, 0])
    HFE.set_title('HFE')
    HFE.step(time, q_plot[:, 1])
    KFE.set_title('KFE')
    KFE.step(time, q_plot[:, 2])
    KFE.set_xlabel('Time (s)', fontsize=14)    
    # joint velocities
    fig2, (HAA, HFE, KFE) = plt.subplots(3, 1, sharex=True)
    fig2.suptitle(str(contact[0:2])+ ' joint velocities (rad/s)')
    HAA.set_title('HAA')
    HAA.step(time, qdot_plot[:, 0])
    HFE.set_title('HFE')
    HFE.step(time, qdot_plot[:, 1])
    KFE.set_title('KFE')
    KFE.step(time, qdot_plot[:, 2])
    KFE.set_xlabel('Time (s)', fontsize=14)   
    # joint accelerations
    fig3, (HAA, HFE, KFE) = plt.subplots(3, 1, sharex=True)
    fig3.suptitle(str(contact[0:2])+ ' joint accelerations (rad/s^2)')
    HAA.set_title('HAA')
    HAA.step(time, qddot_plot[:, 0])
    HFE.set_title('HFE')
    HFE.step(time, qddot_plot[:, 1])
    KFE.set_title('KFE')
    KFE.step(time, qddot_plot[:, 2])
                  

In [ ]:
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model import CentroidalPlusLegKinematicsCasadiModel
import conf_solo12_trot_step_adjustment_full_kinematics as conf
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
from utils import interpolate_one_step
import pinocchio as pin
import numpy as np
import pybullet
import time

# get timings
dt = conf.dt
dt_ctrl = conf.dt_ctrl
N_ctrl =  int(dt/dt_ctrl)
# joint PD gains
Kp = 2*np.eye(12)
Kd = 0.2*np.eye(12)
# save torques
tau_feedfwd = []
tau_feedbck = []
tau_total = []
# get end-effector jacobian functions
ee_jacobians = model_nom.casadi_model.ee_jacobians
pybullet.configureDebugVisualizer(pybullet.COV_ENABLE_GUI,0)
# logger = pybullet.startStateLogging(pybullet.STATE_LOGGING_VIDEO_MP4, 'trot_nom.mp4')
# sinulation loop
for traj_idx in range(conf.N-1):
    # get desired generalized position
    q_des = np.concatenate(
        [
         x_nom[traj_idx, 9:12],
         np.array(
    model_nom.casadi_model.q_plus(x_warmstart[traj_idx][12:16], x_nom[traj_idx, 12:15]*dt)
         ).squeeze(),
         x_nom[traj_idx, 15:27]
        ]
    )
    q_des_next = np.concatenate(
        [
         x_nom[traj_idx+1, 9:12],
         np.array(
    model_nom.casadi_model.q_plus(x_warmstart[traj_idx+1][12:16], x_nom[traj_idx+1, 12:15]*dt)
         ).squeeze(),
         x_nom[traj_idx+1, 15:27]
        ]
    )
    # get desired generalized velocity
    v_des = x_nom[traj_idx, 27:]
    v_des_next = x_nom[traj_idx+1, 27:]
    # get desired generalized acceleration
    a_des = u_nom[traj_idx, 12:]
    a_des_next = u_nom[traj_idx+1, 12:]
    # get desired contact forces
    f_des = u_nom[traj_idx, :12]
    f_des_next = u_nom[traj_idx+1, :12]
    # interpolate solution for low-level torque control
    q_interpol, qdot_interpol, qddot_interpol, f_interpol = interpolate_one_step(
        rmodel, dt, dt_ctrl,
        q_des, q_des_next,
        v_des, v_des_next,
        a_des, a_des_next,
        f_des, f_des_next
    )
    # torque control loop
    for ctrl_time_idx in range(N_ctrl):
        q_interpol_k = q_interpol[ctrl_time_idx]
        # compute J.T @ f
        Jtf = ee_jacobians[1](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_idx, 0:3]) +\
              ee_jacobians[0](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_idx, 3:6]) +\
              ee_jacobians[3](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_idx, 6:9]) +\
              ee_jacobians[2](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_idx, 9:12])
        # compute inverse dynamics
        tau_ff = np.array(
            model_nom.casadi_model.rnea(
                q_interpol_k, qdot_interpol[ctrl_time_idx], qddot_interpol[ctrl_idx]
            )
        )[6:].squeeze() - np.array(Jtf)[6:].squeeze()
        # compute feedback torques
        q_meas, qdot_meas = simulator.robot.get_state()
        tau_fb = Kp @ (q_interpol[ctrl_time_idx, 7:] - q_meas[7:]) + \
                 Kd @ (qdot_interpol[ctrl_time_idx, 6:] - qdot_meas[6:])
        # tau = t_ff + K_p @ (q_d - q_meas) + K_d @ (qdot_d - qdot_meas)
        tau_k = tau_ff + tau_fb
        simulator.robot.send_joint_command(tau_k)
        # step simulation
        simulator.env.step(sleep=True)
        # save torques
        tau_feedfwd += [tau_ff]
        tau_feedbck += [tau_fb]
        tau_total += [tau_k]
# pybullet.stopStateLogging(logger)
    # save closed-loop solution
#     X_sol[traj_time_idx+1] = x_sol[0]
#     U_sol[traj_time_idx] = u_sol[0]
    # warm-start solver from the previous solution
#     x_warm_start_N = np.concatenate([x_sol[1:], x_sol[-1].reshape(1, nx)]) #x_sol
#     u_warm_start_N = np.concatenate([u_sol[1:], u_sol[-1].reshape(1, nu)]) #u_sol 

In [ ]:
tau_feedfwd = np.array(tau_feedfwd)
tau_feedbck = np.array(tau_feedbck)
tau_total = np.array(tau_total)
time = np.arange(0, np.round((tau_feedfwd.shape[0])*dt_ctrl, 2), dt_ctrl)
# plot torques
for contact in conf.ee_frame_names:
    plt.rc('font', family ='serif')
    contact_name = contact[0:2]
    if contact_name == 'FL':
        tau_ff_plot = tau_feedfwd[:, 0:3]
        tau_fb_plot = tau_feedbck[:, 0:3]
        tau_total_plot = tau_total[:, 0:3]
    elif contact_name == 'FR':
        tau_ff_plot = tau_feedfwd[:, 3:6]
        tau_fb_plot = tau_feedbck[:, 3:6]
        tau_total_plot = tau_total[:, 3:6]
    elif contact_name == 'HL':
        tau_ff_plot = tau_feedfwd[:, 6:9]
        tau_fb_plot = tau_feedbck[:, 6:9]
        tau_total_plot = tau_total[:, 6:9]
    elif contact_name == 'HR':
        tau_ff_plot = tau_feedfwd[:, 9:12]
        tau_fb_plot = tau_feedbck[:, 9:12]
        tau_total_plot = tau_total[:, 9:12]
    fig1, (HAA, HFE, KFE) = plt.subplots(3, 1, sharex=True)
    fig1.suptitle(str(contact[0:2])+ ' torques (N.m)')
    HAA.set_title('HAA')
    HAA.step(time, tau_ff_plot[:, 0], label=' feedfwd')
    HAA.step(time, tau_fb_plot[:, 0], label=' feedback')
    HAA.step(time, tau_total_plot[:, 0], label=' total')
    HAA.legend()
    HFE.set_title('HFE')
    HFE.step(time, tau_ff_plot[:, 1], label=' feedfwd')
    HFE.step(time, tau_fb_plot[:, 1], label=' feedback')
    HFE.step(time, tau_total_plot[:, 1], label=' total')
    KFE.set_title('KFE')
    KFE.step(time, tau_ff_plot[:, 2], label=' feedfwd')
    KFE.step(time, tau_fb_plot[:, 2], label=' feedback')
    KFE.step(time, tau_total_plot[:, 2], label=' total')